# 7. Reprezentacja danych

In [38]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as rmse

reg = LinearRegression()


alldata = pd.read_csv(
    'flats.tsv', header=0, sep='\t', skipinitialspace=True,
     usecols=['cena', 'Powierzchnia w m2', 'Liczba pokoi', 'Piętro' ,'Typ zabudowy', 'opis', 'Forma kuchni', "Miejsce parkingowe"])

# MODEL BEZ POPRAWEK

X = alldata[[c for c in alldata.columns if c not in ['opis', 'cena', 'Miejsce parkingowe', 'Forma kuchni', 'Typ zabudowy', 'Piętro']]]
y = alldata['cena']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 6)
lin_reg = reg.fit(X_train, y_train)
score = lin_reg.score(X_test, y_test)
prediction = lin_reg.predict(X_test)
print("Model bez poprawek")
print("RMSE:", rmse(y_test, prediction, squared=False))
print("Score:", score)
print()

# MODEL PORPAWIONY 

# zamiana "parter" na piętro 0
alldata['Piętro'] = alldata['Piętro'].apply(lambda x: 0 if x == 'parter' else x)
# usunięcie mieszkań, które są na poddaszu
alldata['Piętro'] = alldata['Piętro'].apply(pd.to_numeric, errors='coerce') 
alldata = alldata.dropna()
# Utowrzenie wartości bool'owskich (False gdy nie ma jakiegokolwiek typu miejsca parkingowego)
alldata['Miejsce parkingowe'] = alldata['Miejsce parkingowe'].apply(lambda x: True if x in alldata['Miejsce parkingowe'].unique()[1:] else False)
# Utworzenie wartości kategorycznych
alldata = pd.get_dummies(alldata, columns=['Typ zabudowy', 'Forma kuchni'])
# Utowrzenie wartości tekstowych
alldata['Czy przystanek?'] = alldata['opis'].apply(lambda x: True if 'przystanek' in x.lower() else False)

# usunięcie mieszkań, których cena wynosi więcej niż 600 tys. zł
alldata = alldata.drop(alldata[alldata['cena'] > 6*10**5].index)

# normalizacja wartości numerycznych
scaler = MinMaxScaler()
cols_to_norm = [c for c in alldata.columns if c in ['cena', 'Powierzchnia w m2', 'Liczba pokoi', 'Piętro']]
alldata[cols_to_norm] = scaler.fit_transform(alldata[cols_to_norm])

# podział na zbiór treningowy i testowy
X = alldata[[c for c in alldata.columns if c not in ['opis', 'cena']]]
y = alldata['cena']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 6)
lin_reg = reg.fit(X_train, y_train)
score = lin_reg.score(X_test, y_test)
prediction = lin_reg.predict(X_test)
print("Model z poprawkami")
print("RMSE:", rmse(y_test, prediction, squared=False))
print("Score:", score)

Model bez poprawek
RMSE: 226959.08748825188
Score: 0.37802766803233157

Model z poprawkami
RMSE: 0.09398725984056977
Score: 0.6614382923654856
